In [1]:
import torch
from torch.autograd import Variable
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline
torch.manual_seed(2)

In [37]:
X = torch.cuda.FloatTensor([[0,0], [0,1], [1,0], [1,1]])
Y = torch.cuda.FloatTensor([0, 1, 1, 0]).view(-1,1)

In [26]:
class XOR(nn.Module):
    def __init__(self, input_dim = 2, output_dim=1):
        super(XOR, self).__init__()
        self.lin1 = nn.Linear(input_dim, 4)
        self.lin2 = nn.Linear(4, 4)
        self.lin3 = nn.Linear(4, 2)
        self.lin4 = nn.Linear(2, output_dim, bias=False)
    
    def forward(self, x):
        x = self.lin1(x)
        x = F.tanh(x)
        x = self.lin2(x)
        x = F.tanh(x)
        x = self.lin3(x)
        x = F.tanh(x)
        x = self.lin4(x)
        return x

In [9]:
use_cuda = torch.cuda.is_available()
device = torch.device("cuda" if use_cuda else "cpu")

In [27]:
model = XOR().to(device)
print(model)
from torchsummary import summary
summary(model, (2,2))

XOR(
  (lin1): Linear(in_features=2, out_features=4, bias=True)
  (lin2): Linear(in_features=4, out_features=4, bias=True)
  (lin3): Linear(in_features=4, out_features=2, bias=True)
  (lin4): Linear(in_features=2, out_features=1, bias=False)
)
----------------------------------------------------------------
        Layer (type)               Output Shape         Param #
            Linear-1                 [-1, 2, 4]              12
            Linear-2                 [-1, 2, 4]              20
            Linear-3                 [-1, 2, 2]              10
            Linear-4                 [-1, 2, 1]               2
Total params: 44
Trainable params: 44
Non-trainable params: 0
----------------------------------------------------------------
Input size (MB): 0.00
Forward/backward pass size (MB): 0.00
Params size (MB): 0.00
Estimated Total Size (MB): 0.00
----------------------------------------------------------------


/usr/local/lib/python3.7/dist-packages/torch/nn/functional.py:1698: UserWarning: nn.functional.tanh is deprecated. Use torch.tanh instead.
  warnings.warn("nn.functional.tanh is deprecated. Use torch.tanh instead.")


In [28]:
def weights_init(model):
    for m in model.modules():
        if isinstance(m, nn.Linear):
            # initialize the weight tensor, here we use a normal distribution
            m.weight.data.normal_(0, 1)

weights_init(model)

In [29]:
loss_func = nn.L1Loss()

In [30]:
optimizer = optim.SGD(model.parameters(), lr=0.02, momentum=0.9)


In [39]:
epochs = 2001
steps = X.size(0)
for i in range(epochs):
    for j in range(steps):
        data_point = np.random.randint(X.size(0))
        x_var = Variable(X[data_point], requires_grad=False).to(device)
        y_var = Variable(Y[data_point], requires_grad=False).to(device)
        
        optimizer.zero_grad()
        y_hat = model(x_var)
        loss = loss_func.forward(y_hat, y_var)
        loss.backward()
        optimizer.step()
        
    if i % 50 == 0:
        print( "Epoch: {0}, Loss: {1}, ".format(i, loss.data.item()))

/usr/local/lib/python3.7/dist-packages/torch/nn/functional.py:1698: UserWarning: nn.functional.tanh is deprecated. Use torch.tanh instead.
  warnings.warn("nn.functional.tanh is deprecated. Use torch.tanh instead.")


Epoch: 0, Loss: 1.11472749710083, 
Epoch: 50, Loss: 0.9436802864074707, 
Epoch: 100, Loss: 0.7876731157302856, 
Epoch: 150, Loss: 0.16769593954086304, 
Epoch: 200, Loss: 0.20734597742557526, 
Epoch: 250, Loss: 0.17884749174118042, 
Epoch: 300, Loss: 0.01656484603881836, 
Epoch: 350, Loss: 0.1280215084552765, 
Epoch: 400, Loss: 0.768085241317749, 
Epoch: 450, Loss: 0.055534958839416504, 
Epoch: 500, Loss: 0.07249271869659424, 
Epoch: 550, Loss: 0.30090516805648804, 
Epoch: 600, Loss: 0.0490015372633934, 
Epoch: 650, Loss: 0.32353487610816956, 
Epoch: 700, Loss: 0.12848107516765594, 
Epoch: 750, Loss: 0.0681074857711792, 
Epoch: 800, Loss: 0.2741852402687073, 
Epoch: 850, Loss: 0.17261531949043274, 
Epoch: 900, Loss: 0.21547703444957733, 
Epoch: 950, Loss: 0.0013346076011657715, 
Epoch: 1000, Loss: 0.017416102811694145, 
Epoch: 1050, Loss: 0.07490045577287674, 
Epoch: 1100, Loss: 0.20497757196426392, 
Epoch: 1150, Loss: 0.05127608776092529, 
Epoch: 1200, Loss: 0.31572747230529785, 
Epoch